In [ ]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 120.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 111.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 113.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.5 MB/s eta 0:00:00


In [ ]:
!pip install -q imbalanced-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.4/238.4 kB 5.5 MB/s eta 0:00:00


In [ ]:
import os
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gc
from tqdm import tqdm
from shutil import copy2
from PIL import Image

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img
from tensorflow.keras.applications import VGG16

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight

from imblearn.over_sampling import SMOTE

In [ ]:
! pip install -q kaggle
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"willyho11","key":"8b714a4157df8205fddcca04810e62bf"}'}

In [ ]:
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d andrewmvd/ocular-disease-recognition-odir5k

Dataset URL: https://www.kaggle.com/datasets/andrewmvd/ocular-disease-recognition-odir5k
License(s): other
 99% 1.60G/1.62G [00:01<00:00, 1.22GB/s]
100% 1.62G/1.62G [00:01<00:00, 1.22GB/s]


In [ ]:
!unzip -q ocular-disease-recognition-odir5k.zip -d ODIR-5K

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 1. Preprocessing Label
import pandas as pd
import ast

df = pd.read_csv("/content/ODIR-5K/full_df.csv")

# Konversi label ke bentuk readable
def simplify_label(label_list):
    label_map = {
        'N': 'Normal',
        'D': 'Diabetes',
        'G': 'Glaucoma',
        'C': 'Cataract',
        'A': 'AMD',
        'H': 'Hypertension',
        'M': 'Myopia',
        'O': 'Others'
    }
    if not isinstance(label_list, list):
        label_list = ast.literal_eval(label_list)
    return label_map.get(label_list[0], 'Unknown')

df['Label'] = df['labels'].apply(simplify_label)

# Gabung Left dan Right Fundus
data = []
for _, row in df.iterrows():
    if pd.notna(row['Left-Fundus']):
        data.append((row['Left-Fundus'], row['Label']))
    if pd.notna(row['Right-Fundus']):
        data.append((row['Right-Fundus'], row['Label']))

df_new = pd.DataFrame(data, columns=['image', 'label'])

# 2. Split 80/10/10
from sklearn.model_selection import train_test_split

trainval_df, test_df = train_test_split(df_new, test_size=0.1, stratify=df_new['label'], random_state=0)
train_df, val_df = train_test_split(trainval_df, test_size=0.1111, stratify=trainval_df['label'], random_state=0)

In [ ]:
# 3. Copy ke Folder
import os
from shutil import copy2
from tqdm import tqdm

src_folder = "/content/ODIR-5K/preprocessed_images"
target_base = "/content/ODIR-5K/split"

for split, dataframe in [('train', train_df), ('val', val_df), ('test', test_df)]:
    for label in df_new['label'].unique():
        os.makedirs(f"{target_base}/{split}/{label}", exist_ok=True)

    for _, row in tqdm(dataframe.iterrows(), total=len(dataframe)):
        filename = row['image']
        label = row['label']
        src = os.path.join(src_folder, filename)
        dst = os.path.join(target_base, split, label, filename)
        if os.path.exists(src):
            copy2(src, dst)

In [ ]:
# --- 4. Load Images into Array ---
src_folder = "/content/ODIR-5K/preprocessed_images"

train_images, train_labels = [], []

for _, row in tqdm(train_df.iterrows(), total=len(train_df)):
    filepath = os.path.join(src_folder, row['image'])
    if os.path.exists(filepath):
        img = load_img(filepath, target_size=(224, 224))
        img_array = img_to_array(img) / 255.0
        train_images.append(img_array)
        train_labels.append(row['label'])

X_train = np.array(train_images)
y_train = np.array(train_labels)

le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)

100%|██████████| 10226/10226 [00:21<00:00, 469.15it/s]


In [ ]:
# --- 5. Apply SMOTE ---
X_flat = X_train.reshape((X_train.shape[0], 224*224*3))

sm = SMOTE(random_state=42)
X_smote, y_smote = sm.fit_resample(X_flat, y_train_enc)

X_smote_img = X_smote.reshape((-1, 224, 224, 3))

y_smote_label = le.inverse_transform(y_smote)

In [ ]:
# --- 6. Save SMOTE Results to Folder ---
smote_dir = "/content/ODIR-5K/SMOTE_balanced"
for cls in np.unique(y_smote_label):
    os.makedirs(os.path.join(smote_dir, cls), exist_ok=True)

for i in range(len(X_smote_img)):
    img = array_to_img(X_smote_img[i])
    label = y_smote_label[i]
    save_path = os.path.join(smote_dir, label, f"smote_{i}.jpg")
    img.save(save_path)

del X_smote_img, y_smote_label
gc.collect()

print("✅ Semua gambar hasil SMOTE telah disimpan di:", smote_dir)

✅ Semua gambar hasil SMOTE telah disimpan di: /content/ODIR-5K/SMOTE_balanced
